```
locations = [
    [],  # 점심식사 (인덱스 % 5 == 0)
    [23, "경찰혼", 37.5263408571, 126.9006431406],  # 활동
    [],  # 저녁식사 (인덱스 % 5 == 2)
    [30, "고덕동 생태경관 보전지역", 37.5670267098, 127.1493110052],  # 활동
    [],  # 숙소 (인덱스 % 5 == 4)
    [],  # 점심식사
    [39, "과학동아천문대", 37.5337300099, 126.963378882],  # 활동
    [],  # 저녁식사
    [63, "구강사", 37.5449150194, 127.1288607188],  # 활동
    [],  # 숙소
    [],  # 점심식사
    [66, "구로구 반려견 놀이터", 37.5082258503, 126.8757403642],  # 활동
    [],  # 저녁식사
    [67, "구로기계공구단지", 37.5041347264, 126.8795151353],  # 활동
    [],  # 숙소
    [],  # 점심식사
    [78, "국악사랑", 37.5829218735, 126.9833340052],  # 활동
    [],  # 저녁식사
    [139, "대림어린이공원", 37.4914005142, 126.9030355727],  # 활동
    [],  # 숙소
    [],  # 점심식사
    [250, "배봉산", 37.5807819121, 127.0643739039],  # 활동
]
```

```
[2854402, '당산오돌 본점', 37.5276156945, 126.8998320674]
[23, '경찰혼', 37.5263408571, 126.9006431406]
[2839476, '33Acre', 37.5632188287, 127.1498471762]
[30, '고덕동 생태경관 보전지역', 37.5670267098, 127.1493110052]
[1781984, '가바나모텔', 37.5972257905, 127.1388777598]
[2608909, '[백년가게]창성옥', 37.5364601949, 126.9604179636]
[39, '과학동아천문대', 37.5337300099, 126.963378882]
[2809790, '송월냉면', 37.5415348675, 127.1294774259]
[63, '구강사', 37.5449150194, 127.1288607188]
[3080615, '아르고호텔', 37.5362439489, 127.1368741254]
[2774271, '뒤뜰', 37.5051288955, 126.8705548127]
[66, '구로구 반려견 놀이터', 37.5082258503, 126.8757403642]
[2653716, '광명수산', 37.5057012674, 126.8826675817]
[67, '구로기계공구단지', 37.5041347264, 126.8795151353]
[136332, '유로파라텔', 37.5001992975, 126.8921252084]
[1154331, '파툼(FATUM)', 37.5832023745, 126.9821368841]
[78, '국악사랑', 37.5829218735, 126.9833340052]
[2682646, '형제꼬치집', 37.4892784837, 126.8999609122]
[139, '대림어린이공원', 37.4914005142, 126.9030355727]
[2505908, '롯데시티호텔 구로', 37.4850844673, 126.8966252225]
[2842950, '천호낙지', 37.5789683, 127.0680236226]
[250, '배봉산', 37.5807819121, 127.0643739039]
```

In [13]:
import pandas as pd
from geopy.distance import geodesic
import os
from tqdm import tqdm

# CSV 파일 경로
attractions_path = "../csv/attractions.csv"

# 데이터 로드
attractions = pd.read_csv(attractions_path)

# 조건에 맞는 데이터 필터링
lunch_dinner_data = attractions[attractions['content_type_id'] == 39]  # 점심/저녁
lodging_data = attractions[attractions['content_type_id'] == 32]      # 숙소

# 가장 가까운 위치를 찾는 함수
def find_closest(target_lat, target_lon, data):
    target_location = (target_lat, target_lon)
    min_distance = float("inf")
    closest_row = None

    for _, row in data.iterrows():
        row_location = (row["latitude"], row["longitude"])
        distance = geodesic(target_location, row_location).meters  # 거리 계산
        if distance < min_distance:
            min_distance = distance
            closest_row = row

    return closest_row

# 빈 리스트를 채우는 함수
def fill_empty_locations(locations):
    for i in range(len(locations)):
        if locations[i] == []:  # 빈 리스트
            # 점심식사 (인덱스 % 5 == 0)
            if i % 5 == 0 and i + 1 < len(locations):
                next_lat, next_lon = locations[i + 1][2], locations[i + 1][3]
                closest_row = find_closest(next_lat, next_lon, lunch_dinner_data)
                locations[i] = [
                    closest_row["content_id"],
                    closest_row["title"],
                    closest_row["latitude"],
                    closest_row["longitude"],
                ]
            # 저녁식사 (인덱스 % 5 == 2)
            elif i % 5 == 2 and i + 1 < len(locations):
                next_lat, next_lon = locations[i + 1][2], locations[i + 1][3]
                closest_row = find_closest(next_lat, next_lon, lunch_dinner_data)
                locations[i] = [
                    closest_row["content_id"],
                    closest_row["title"],
                    closest_row["latitude"],
                    closest_row["longitude"],
                ]
            # 숙소 (인덱스 % 5 == 4)
            elif i % 5 == 4 and i - 1 >= 0:
                prev_lat, prev_lon = locations[i - 1][2], locations[i - 1][3]
                closest_row = find_closest(prev_lat, prev_lon, lodging_data)
                locations[i] = [
                    closest_row["content_id"],
                    closest_row["title"],
                    closest_row["latitude"],
                    closest_row["longitude"],
                ]
    return locations

# 입력 및 출력 디렉토리 설정
input_dir = "./outputS"
output_dir = "./my_courses"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# ./output/ 디렉토리의 모든 txt 파일 처리
for file_name in tqdm(os.listdir(input_dir), desc="Processing files"):
    if file_name.endswith(".txt"):  # TXT 파일만 처리
        file_path = os.path.join(input_dir, file_name)

        # travel_plans 로드
        with open(file_path, "r", encoding="utf-8") as f:
            file_content = f.read()
            exec(file_content)  # travel_plans 리스트 실행

        # travel_plans 내부의 각 locations에 대해 fill_empty_locations 수행
        for locations in tqdm(travel_plans, desc=f"Processing {file_name} locations", leave=False):
            fill_empty_locations(locations)

        # 처리된 travel_plans를 문자열로 변환하여 저장
        output_file_path = os.path.join(output_dir, file_name)
        with open(output_file_path, "w", encoding="utf-8") as f:
            f.write(str(travel_plans))

        print(f"{file_name} processed and saved to {output_file_path}")


Processing 1_attraction_response.txt locations:   0%|       | 0/7 [00:00<?, ?it/s]
Processing 1_attraction_response.txt locations:  14%|▏| 1/7 [00:24<02:25, 24.25s/i
Processing 1_attraction_response.txt locations:  29%|▎| 2/7 [00:48<02:00, 24.19s/i
Processing 1_attraction_response.txt locations:  43%|▍| 3/7 [01:12<01:36, 24.12s/i
Processing 1_attraction_response.txt locations:  57%|▌| 4/7 [01:36<01:12, 24.22s/i
Processing 1_attraction_response.txt locations:  71%|▋| 5/7 [02:00<00:48, 24.20s/i
Processing 1_attraction_response.txt locations:  86%|▊| 6/7 [02:25<00:24, 24.21s/i
Processing 1_attraction_response.txt locations: 100%|█| 7/7 [02:49<00:00, 24.13s/i
Processing files:  10%|██▋                        | 1/10 [02:49<25:22, 169.17s/it]

1_attraction_response.txt processed and saved to ./my_courses\1_attraction_response.txt



Processing 2_attraction_response.txt locations:   0%|       | 0/7 [00:00<?, ?it/s]
                                                                                  

2_attraction_response.txt processed and saved to ./my_courses\2_attraction_response.txt



Processing 31_attraction_response.txt locations:   0%|      | 0/7 [00:00<?, ?it/s]
Processing 31_attraction_response.txt locations:  14%|▏| 1/7 [00:23<02:23, 23.95s/
Processing 31_attraction_response.txt locations:  29%|▎| 2/7 [00:48<02:01, 24.21s/
Processing 31_attraction_response.txt locations:  43%|▍| 3/7 [01:12<01:37, 24.39s/
Processing 31_attraction_response.txt locations:  57%|▌| 4/7 [01:31<01:06, 22.16s/
Processing 31_attraction_response.txt locations:  71%|▋| 5/7 [01:50<00:41, 20.94s/
Processing 31_attraction_response.txt locations:  86%|▊| 6/7 [02:09<00:20, 20.20s/
Processing 31_attraction_response.txt locations: 100%|█| 7/7 [02:28<00:00, 20.04s/
Processing files:  30%|████████▍                   | 3/10 [05:18<11:33, 99.02s/it]

...

36_attraction_response.txt processed and saved to ./my_courses\36_attraction_response.txt



Processing 37_attraction_response.txt locations:   0%|      | 0/7 [00:00<?, ?it/s]
                                                                                  

37_attraction_response.txt processed and saved to ./my_courses\37_attraction_response.txt



Processing files: 100%|███████████████████████████| 10/10 [14:07<00:00, 84.76s/it]

6_attraction_response.txt processed and saved to ./my_courses\6_attraction_response.txt
